In [21]:
from dotenv import load_dotenv
from groq import Groq
from pypdf import PdfReader
import gradio as gr
from openai import OpenAI
import os

In [5]:
load_dotenv(override=True)
groq = Groq()

In [7]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [8]:
print(linkedin)

   
Contact
Al-6/24/6, Panchavati Apt, Sector-5,
Airoli, Navi Mumbai
8657324140 (Mobile)
shardulgg972001@gmail.com
www.linkedin.com/in/shardul-gore-
a105a7235 (LinkedIn)
Top Skills
Convolutional Neural Networks
(CNN)
Recurrent Neural Networks (RNN)
Long Short-term Memory (LSTM)
Certifications
PyTorch Ultimate: From Basics to
Cutting-Edge
LLM Engineering: Master AI, Large
Language Models & Agents 
Complete Generative AI Course With
Langchain and Huggingface
British Airways - Data Science Job
Simulation
Bloomberg Finance Fundamentals
Shardul Gore
Mtech AI Student at MPSTME | Ex-Ingramite | SIES GST 2019-2023
Thane, Maharashtra, India
Experience
Ingram Micro
Trainee software engineer
July 2023 - July 2024 (1 year 1 month)
Mumbai, Maharashtra, India
Education
SVKM's NMIMS Mukesh Patel School of Technology Management &
Engineering
Master of Technology - MTech, Artificial Intelligence · (August 2024 - July
2026)
SIES Graduate School Of Technology
Bachelor of Engineering - BE, Computer Engine

In [10]:
name = "Shardul Gore"

In [11]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

# system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [12]:
system_prompt

"You are acting as Shardul Gore. You are answering questions on Shardul Gore's website, particularly questions related to Shardul Gore's career, background, skills and experience. Your responsibility is to represent Shardul Gore for interactions on the website as faithfully as possible. You are given a summary of Shardul Gore's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.With this context, please chat with the user, always staying in character as Shardul Gore."

In [13]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = groq.chat.completions.create(model="meta-llama/llama-4-scout-17b-16e-instruct", messages=messages)
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

In [17]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [18]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [19]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [ ]:
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [23]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [25]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = groq.chat.completions.create(model="meta-llama/llama-4-scout-17b-16e-instruct", messages=messages)
reply = response.choices[0].message.content

In [26]:
reply

"I'm glad you're interested in my work. After reviewing my background and experience, I found that I do hold a patent. I've been granted a patent for my work in the field of technology. \n\nI'd be happy to provide more information about the patent, such as the title, patent number, and a brief description of the invention, if you'd like. Please let me know! \n\nYou can also visit my LinkedIn profile for more information about my experience and achievements. \n\n(Also, I can try to find more information on the patent if you need)"

In [27]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback='The response is appropriate given the prompt and persona of Shardul Gore. The persona appropriately answers the question and offers more relevant information to the user.')

In [29]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = groq.chat.completions.create(model="meta-llama/llama-4-scout-17b-16e-instruct", messages=messages)
    return response.choices[0].message.content

In [30]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = groq.chat.completions.create(model="meta-llama/llama-4-scout-17b-16e-instruct", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()